In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'mgi_symbol':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
    return values  


def biomartHumanOrthologs(mgi_symbol):
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'mgi_symbol':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)

    return values


def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'mgi_id':[mgi_id]}                     # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)

    return values  


def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    lines = response.text.split("\n")
    lines = list(filter(lambda l: l != "",lines))
    values = []
    for l in lines:
        temp = []
        raw_values = l.split("\t")
        values.append(raw_values)
        
    return values


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and open workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and open workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify the desired cell
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    # Save the file
    workbook.save(file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook('../ifpan-GR-database-papers.xlsx')
ws = wb['NPSCs']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]

## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [5]:
# LISTS

dictionary = {}
ls_geneDictionaries = []

ls_notResponse = []

ls_notResponse_after = []


In [6]:
# DICTIONARY

for i in range(len(publication_geneName)):
    gene_name = publication_geneName[i]
    direction_string = direction[i]   # => info
    
    if gene_name != '':

        # VARIABLES
        index = i + 1
        gene_list_number = 26
        gene_list_id =  'all_sig_genes_26606517'      # => cluster
        source = 'PMID: 26606517'
        organism = 'mouse'
        print(direction_string)
        alias = ''


        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters(gene_name)
        if not ls_biomartParameters:
            ls_notResponse.append(gene_name)
            print(ls_notResponse)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
        
        
        # ORTHOLOGS
        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





up
{'Index': 1, 'Gene_name': 'Fam107a', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000021750', 'Ensembl_transcript_id': 'ENSMUST00000137133|ENSMUST00000036070|ENSMUST00000121887|ENSMUST00000120411', 'RefSeq_mRNA_id': 'NM_001360338|NM_183187|NM_001360337|NM_001360339', 'Organism': 'mouse', 'HGNC_symbol': 'FAM107A', 'Alias': '', 'Info': 'up'}
up
{'Index': 2, 'Gene_name': 'Hif3a', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000004328', 'Ensembl_transcript_id': 'ENSMUST00000037762|ENSMUST00000108492|ENSMUST00000139224|ENSMUST00000153833', 'RefSeq_mRNA_id': 'NM_001162950|NM_016868', 'Organism': 'mouse', 'HGNC_symbol': 'HIF3A', 'Alias': '', 'Info': 'up'}
up
{'Index': 3, 'Gene_name': 'Cftr', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000041301', 'En

['Fam101b']
{'Index': 21, 'Gene_name': 'Fam101b', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': '', 'Ensembl_transcript_id': '', 'RefSeq_mRNA_id': '', 'Organism': 'mouse', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
up
{'Index': 22, 'Gene_name': 'Tprn', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000048707', 'Ensembl_transcript_id': 'ENSMUST00000141509|ENSMUST00000137361|ENSMUST00000114336|ENSMUST00000155738', 'RefSeq_mRNA_id': 'NM_175286', 'Organism': 'mouse', 'HGNC_symbol': 'TPRN', 'Alias': '', 'Info': 'up'}
up
{'Index': 23, 'Gene_name': 'Klf9', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000033863', 'Ensembl_transcript_id': 'ENSMUST00000236357|ENSMUST00000036884', 'RefSeq_mRNA_id': 'NM_010638', 'Organism': 'mouse', 'HGNC_symbol': 'KLF9', 'Alias': '', 

{'Index': 43, 'Gene_name': 'Bcat1', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000030268', 'Ensembl_transcript_id': 'ENSMUST00000032402|ENSMUST00000048252|ENSMUST00000154611|ENSMUST00000145911|ENSMUST00000149769|ENSMUST00000136819|ENSMUST00000155693|ENSMUST00000204138|ENSMUST00000111742|ENSMUST00000123930', 'RefSeq_mRNA_id': 'NM_001024468|NM_007532', 'Organism': 'mouse', 'HGNC_symbol': 'BCAT1', 'Alias': '', 'Info': 'up'}
up
{'Index': 44, 'Gene_name': 'Kcnj12', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000042529', 'Ensembl_transcript_id': 'ENSMUST00000041944|ENSMUST00000108717|ENSMUST00000089184', 'RefSeq_mRNA_id': 'NM_010603|NM_001267593', 'Organism': 'mouse', 'HGNC_symbol': 'KCNJ18', 'Alias': '', 'Info': 'up'}
up
{'Index': 45, 'Gene_name': 'Chst2', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Sou

{'Index': 63, 'Gene_name': 'Ociad2', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000029153', 'Ensembl_transcript_id': 'ENSMUST00000201908|ENSMUST00000202012|ENSMUST00000087195|ENSMUST00000200830|ENSMUST00000200821|ENSMUST00000200776', 'RefSeq_mRNA_id': 'NM_026950|NM_001310636', 'Organism': 'mouse', 'HGNC_symbol': 'OCIAD2', 'Alias': '', 'Info': 'up'}
up
{'Index': 64, 'Gene_name': 'Fkbp14', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000038074', 'Ensembl_transcript_id': 'ENSMUST00000123832|ENSMUST00000143312|ENSMUST00000154907|ENSMUST00000046520|ENSMUST00000155047|ENSMUST00000117375|ENSMUST00000141757', 'RefSeq_mRNA_id': 'NM_001363793|NM_001363792|NM_153573', 'Organism': 'mouse', 'HGNC_symbol': 'FKBP14', 'Alias': '', 'Info': 'up'}
up
{'Index': 65, 'Gene_name': 'Mt1', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26

{'Index': 84, 'Gene_name': 'Lgalsl', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000042363', 'Ensembl_transcript_id': 'ENSMUST00000154300|ENSMUST00000047028|ENSMUST00000132928|ENSMUST00000144441', 'RefSeq_mRNA_id': 'NM_173752', 'Organism': 'mouse', 'HGNC_symbol': 'LGALSL', 'Alias': '', 'Info': 'up'}
up
{'Index': 85, 'Gene_name': 'Zfp189', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000039634', 'Ensembl_transcript_id': 'ENSMUST00000107696|ENSMUST00000042964', 'RefSeq_mRNA_id': 'NM_001289901|NM_145547', 'Organism': 'mouse', 'HGNC_symbol': 'ZNF189', 'Alias': '', 'Info': 'up'}
up
{'Index': 86, 'Gene_name': 'Per2', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000055866', 'Ensembl_transcript_id': 'ENSMUST00000185298|ENSMUST00000069620', 'RefSeq_mRNA_i

{'Index': 105, 'Gene_name': 'Ddit4', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000020108', 'Ensembl_transcript_id': 'ENSMUST00000020308', 'RefSeq_mRNA_id': 'NM_029083', 'Organism': 'mouse', 'HGNC_symbol': 'DDIT4', 'Alias': '', 'Info': 'up'}
up
{'Index': 106, 'Gene_name': 'Elmo1', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000041112', 'Ensembl_transcript_id': 'ENSMUST00000181429|ENSMUST00000220751|ENSMUST00000221595|ENSMUST00000180626|ENSMUST00000072519|ENSMUST00000180767|ENSMUST00000221067', 'RefSeq_mRNA_id': 'NM_080288', 'Organism': 'mouse', 'HGNC_symbol': 'ELMO1', 'Alias': '', 'Info': 'up'}
up
{'Index': 107, 'Gene_name': 'Mapk4', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000024558', 'Ensembl_transcript_id': 'ENSMUST00000160601|ENSMUST000

{'Index': 125, 'Gene_name': 'Pcdha2', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000104148', 'Ensembl_transcript_id': 'ENSMUST00000115662|ENSMUST00000195590', 'RefSeq_mRNA_id': 'NM_198117', 'Organism': 'mouse', 'HGNC_symbol': 'PCDHA2', 'Alias': '', 'Info': 'down'}
down
{'Index': 126, 'Gene_name': 'Jun', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000052684', 'Ensembl_transcript_id': 'ENSMUST00000107094', 'RefSeq_mRNA_id': 'NM_010591', 'Organism': 'mouse', 'HGNC_symbol': 'JUN', 'Alias': '', 'Info': 'down'}
down
{'Index': 127, 'Gene_name': 'Rassf3', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000025795', 'Ensembl_transcript_id': 'ENSMUST00000219500|ENSMUST00000026902', 'RefSeq_mRNA_id': 'NM_138956', 'Organism': 'mouse', 'HGNC_symbol': 'RASSF3', 

{'Index': 145, 'Gene_name': 'Adamts3', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000043635', 'Ensembl_transcript_id': 'ENSMUST00000198151|ENSMUST00000196507|ENSMUST00000061427|ENSMUST00000163159', 'RefSeq_mRNA_id': 'NM_001081401|NM_177872', 'Organism': 'mouse', 'HGNC_symbol': 'ADAMTS3', 'Alias': '', 'Info': 'down'}
down
{'Index': 146, 'Gene_name': 'Nhs', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000059493', 'Ensembl_transcript_id': 'ENSMUST00000087085|ENSMUST00000081569', 'RefSeq_mRNA_id': 'NM_001290526|NM_001081052', 'Organism': 'mouse', 'HGNC_symbol': 'NHS', 'Alias': '', 'Info': 'down'}
down
{'Index': 147, 'Gene_name': 'Pde1b', 'Gene_list_number': 26, 'Gene_list_index': 'all_sig_genes_26606517', 'Source': 'PMID: 26606517', 'Ensembl_gene_id': 'ENSMUSG00000022489', 'Ensembl_transcript_id': 'ENSMUST00000227955|ENSMUST0000

In [7]:
print(ls_notResponse)

['Fam101b', 'Ctgf', 'Phf15', '1810011O10Rik', 'Ndnl2', 'BC029214', 'Sepp1', 'Mtss1l']


In [8]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb.active
mgi_id = [ws_3.cell(row=i,column=3).value for i in range(3,ws_3.max_row+1)]
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

['1810011o10rik', 50962, 'MGI:1916318']


In [9]:
print (ls_notResponse)

['Fam101b', 'Ctgf', 'Phf15', '1810011O10Rik', 'Ndnl2', 'BC029214', 'Sepp1', 'Mtss1l']


In [11]:
# SECOND DICTIONARY

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_v2[i]
    print(mgi_ID)
    
    if gene_name != '':

        # variables
        alias = ''

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(mgi_ID)
        
        print(ls_biomartParameters)
        
        if not ls_biomartParameters:
            ls_notResponse_after.append(mgi_ID)


        for ls in ls_biomartParameters:
            for j in range(3):
                if len(ls) < (j+1):
                    ls.append('')
            ensembl_gene_id_temp.append(ls[0])
            ensembl_transcript_id_temp.append(ls[1])
            refseq_mrna_temp.append(ls[2])  

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))
         
        
        ls_biomartHumanOrthologs = biomartHumanOrthologs_mgi(mgi_ID)
        if not ls_biomartHumanOrthologs:
            hgnc_symbol = ''
        else:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]       
        
        
        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


Hif3a
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Cftr
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', 

Pdk4
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Nedd9
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', 

Frmpd1
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Mt2
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', 

Fam43a
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Rhob
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693',

Arrdc2
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Dyrk3
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693'

Mgll
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Tlcd2
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', 

Sdc4
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Arid5b
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693',

Mxd4
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Sema4b
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693',

Uaca
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Rprm
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', '

Nhs
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', ''], ['ENSMUSG00000030400', 'ENSMUST00000145039', '']]
Pde1b
[['ENSMUSG00000030400', 'ENSMUST00000062831', 'NM_007949'], ['ENSMUSG00000030400', 'ENSMUST00000136055', ''], ['ENSMUSG00000030400', 'ENSMUST00000108461', ''], ['ENSMUSG00000030400', 'ENSMUST00000108460', 'NM_001363981'], ['ENSMUSG00000030400', 'ENSMUST00000127363', ''], ['ENSMUSG00000030400', 'ENSMUST00000129249', ''], ['ENSMUSG00000030400', 'ENSMUST00000128167', ''], ['ENSMUSG00000030400', 'ENSMUST00000129291', ''], ['ENSMUSG00000030400', 'ENSMUST00000135693', '

In [12]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [13]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [14]:
# Add INFO

def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate

### the number and type of items in the INFO column depends on the source publication


            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'time:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'comparision:' + str(ws_info['Q' + str(j)].value) + '|'
                            + 'strain:' + str(ws_info['R' + str(j)].value))

    # Save the file
    wb_alias.save(info_file_path)



source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'NPSCs'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


fam107a fam107a
hif3a hif3a
cftr cftr
zbtb16 zbtb16
rasgef1c rasgef1c
ptk2b ptk2b
sphkap sphkap
ada ada
kcnt1 kcnt1
kcnn2 kcnn2
thrsp thrsp
tsc22d3 tsc22d3
crip1 crip1
fkbp5 fkbp5
mfsd2a mfsd2a
wnt3 wnt3
gbx2 gbx2
pdk4 pdk4
nedd9 nedd9
dio2 dio2
fam101b fam101b
tprn tprn
klf9 klf9
per1 per1
amigo2 amigo2
gfpt2 gfpt2
map7d2 map7d2
pgbd5 pgbd5
mertk mertk
ctgf ctgf
arl4d arl4d
pgf pgf
ccl2 ccl2
frmpd1 frmpd1
mt2 mt2
kank1 kank1
phf15 phf15
camkk1 camkk1
rhod rhod
crispld2 crispld2
nkd2 nkd2
rhou rhou
bcat1 bcat1
kcnj12 kcnj12
chst2 chst2
cables1 cables1
rcan2 rcan2
cpne7 cpne7
errfi1 errfi1
fam43a fam43a
rhob rhob
rhoj rhoj
bcl2l1 bcl2l1
sesn1 sesn1
plekhf1 plekhf1
adamts9 adamts9
pla2g3 pla2g3
bcl6 bcl6
agt agt
parp1 parp1
hspa12a hspa12a
smox smox
ociad2 ociad2
fkbp14 fkbp14
mt1 mt1
arrdc2 arrdc2
dyrk3 dyrk3
fam13a fam13a
tmem229b tmem229b
htr3a htr3a
1810011o10rik 1810011o10rik
col5a3 col5a3
cdh19 cdh19
adrb2 adrb2
cebpa cebpa
tbc1d10a tbc1d10a
paqr8 paqr8
nfkbia nfkbia
sap30 sap30
pm

In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''